In [17]:
import io
import os
import json
import time

from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
from collections import defaultdict
import tiktoken # for token counting

In [18]:
load_dotenv(override=True)

True

In [19]:
client = OpenAI()

In [20]:
encoding = tiktoken.get_encoding("cl100k_base")

### Helper functions

In [21]:
def json_to_jsonl(input_file, output_file):
    """Converts a JSON file to a JSONL file."""
    # Open JSON file
    f = open(input_file)

    # Returns JSON object as a dictionary
    data = json.load(f)

    # Produce JSONL from JSON
    with open(output_file, "w") as outfile:
        for entry in data:
            json.dump(entry, outfile)
            outfile.write("\n")

In [22]:
def check_file_format(dataset):
    """Checks the format of a dataset."""
    format_errors = defaultdict(int)

    for ex in dataset:
        if not isinstance(ex, dict):
            format_errors["data_type"] += 1
            continue

        messages = ex.get("messages", None)
        if not messages:
            format_errors["missing_messages_list"] += 1
            continue

        for message in messages:
            if "role" not in message or "content" not in message:
                format_errors["message_missing_key"] += 1

            if any(
                k not in ("role", "content", "name", "function_call") for k in message
            ):
                format_errors["message_unrecognized_key"] += 1

            if message.get("role", None) not in (
                "system",
                "user",
                "assistant",
                "function",
            ):
                format_errors["unrecognized_role"] += 1

            content = message.get("content", None)
            function_call = message.get("function_call", None)

            if (not content and not function_call) or not isinstance(content, str):
                format_errors["missing_content"] += 1

        if not any(message.get("role", None) == "assistant" for message in messages):
            format_errors["example_missing_assistant_message"] += 1

    if format_errors:
        print("Found errors:")
        for k, v in format_errors.items():
            print(f"{k}: {v}")
    else:
        print("No errors found!")

In [23]:
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

In [24]:
# Convert JSON to JSONL
json_to_jsonl("teacrafter.json", "output.jsonl")

In [25]:
# Load dataset (https://cookbook.openai.com/examples/chat_finetuning_data_prep)
data_path = "output.jsonl"

with open(data_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

In [26]:
# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 18
First example:
{'role': 'system', 'content': 'This is a customer support chatbot designed to help with inquiries for TeaCrafters, an artisan tea subscription service.'}
{'role': 'user', 'content': 'How do I change my tea preferences for the next shipment?'}
{'role': 'assistant', 'content': "You can update your tea preferences by logging into your account and visiting the 'Subscription Settings' page. Changes must be made at least 7 days before your next shipment."}


In [27]:
# Format validation
check_file_format(dataset)

No errors found!


In [28]:
# Cost estimations
conversation_length = [] # Get the length of the conversation

for msg in dataset:
    messages = msg["messages"]
    conversation_length.append(num_tokens_from_messages(messages))

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096
TARGET_EPOCHS = 5
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)

if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(
    min(MAX_TOKENS_PER_EXAMPLE, length) for length in conversation_length
)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"Train for {n_epochs} epochs and be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

num_tokens = n_epochs * n_billing_tokens_in_dataset

# Pricing: https://platform.openai.com/docs/pricing
# gpt-4o-mini-2024-07-18: $3.00 / 1M tokens
cost = (num_tokens / 1e6) * 3
print(f"Estimated cost: {cost}")

Dataset has ~1437 tokens that will be charged for during training
Train for 5 epochs and be charged for ~7185 tokens
Estimated cost: 0.021555


In [29]:
# Upload file once all validations are successful!
training_file = client.files.create(
    file=open("output.jsonl", "rb"), purpose="fine-tune"
)
print(training_file.id)

file-9JF9pUDLi453Tqa1tkd8hJ


In [30]:
# Start the fine-tuning job (may take some time to complete, based on model and dataset)
job = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-4o-mini-2024-07-18",
    hyperparameters={
        "n_epochs": n_epochs,
    },
)

In [31]:
print(job.id)

ftjob-GAPULfJ44qMeS1ZpXfNXTcee


In [32]:
# Retrieve the state of a fine-tune
# Status field can contain: running || succeeded || failed, etc.
fine_tuned_model = None
while not fine_tuned_model:
    state = client.fine_tuning.jobs.retrieve(job.id)
    print("Current status:", state.status)
    # Done: get fine-tuned model
    if state.status == "succeeded":
        fine_tuned_model = state.fine_tuned_model
        print("Fine-tuned-model:", fine_tuned_model)
    # Wait for 30 seconds before checking again
    else:
        print("Wait for 30 seconds before checking again")
        time.sleep(30)


Current status: validating_files
Wait for 30 seconds before checking again
Current status: validating_files
Wait for 30 seconds before checking again
Current status: validating_files
Wait for 30 seconds before checking again
Current status: validating_files
Wait for 30 seconds before checking again
Current status: validating_files
Wait for 30 seconds before checking again
Current status: validating_files
Wait for 30 seconds before checking again
Current status: running
Wait for 30 seconds before checking again
Current status: running
Wait for 30 seconds before checking again
Current status: running
Wait for 30 seconds before checking again
Current status: running
Wait for 30 seconds before checking again
Current status: running
Wait for 30 seconds before checking again
Current status: running
Wait for 30 seconds before checking again
Current status: running
Wait for 30 seconds before checking again
Current status: running
Wait for 30 seconds before checking again
Current status: runnin

In [33]:
print(fine_tuned_model)

ft:gpt-4o-mini-2024-07-18:personal::BokouGwm


In [ ]:
# Get response from fine-tuned model
response = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[
        {
            "role": "system",
            "content": "This is a customer support chatbot designed to help with common inquiries.",
            "role": "user",
            "content": "How do I cancel my subscription?",
        }
    ],
)
print("Fine-tuned response:")
print(response.choices[0].message.content)

# Note: Not so good, try 10-15 epochs insteal

Fine-tuned response:
To cancel your subscription, log into your account and visit the 'Subscription Settings' page. Click on 'Cancel Subscription' and follow the prompts. Please note that cancellations must be made at least 24 hours before your next billing cycle.


In [39]:
# Get response from normal model
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=[
        {
            "role": "system",
            "content": "This is a customer support chatbot designed to help with common inquiries.",
            "role": "user",
            "content": "How do I cancel my subscription?",
        }
    ],
)
print("Normal response:")
print(response.choices[0].message.content)

Normal response:
To cancel your subscription, you'll typically need to follow these general steps, though the specific process may vary depending on the service:

1. **Log into Your Account**: Go to the website or app of the service you're subscribed to and log into your account.

2. **Navigate to Account Settings**: Look for a section labeled "Account," "Settings," or "Subscription Management."

3. **Find Subscription Information**: Within the account settings, find the area that contains details about your subscription or billing.

4. **Select Cancel Subscription**: Look for an option to cancel or manage your subscription. This might be labeled as "Cancel Subscription," "End Membership," or similar.

5. **Follow the Prompts**: The service will likely guide you through the cancellation process. You may need to confirm your decision or provide a reason for canceling.

6. **Check for Confirmation**: After canceling, ensure that you receive a confirmation (via email or on-screen) that yo